# Racial Bias Analysis

### Setting up environment and loading Preprocessed Training/Test Data

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import sem
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
xTrain=pd.read_csv('xTrain.csv')
xTest=pd.read_csv('xTest.csv')
yTrain=pd.read_csv('yTrain.csv')
yTest=pd.read_csv('yTest.csv')
xTest.head()
xTrain = xTrain.drop(columns=['Unnamed: 0'])
xTest = xTest.drop(columns=['Unnamed: 0'])

### Creating test sets with only specific races for Bias Analysis

In [3]:
frames = [xTest, yTest]
TestSet = pd.concat(frames, axis=1)
TestSet = TestSet.drop(columns=['Unnamed: 0'])
TestSet.head()

,admission_location,insurance,marital_status,los,sofa_24hours,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT,ASIAN,BLACK/AFRICAN AMERICAN,HISPANIC/LATINO,WHITE,ethnicity,hospital_expire_flag
0,-1.019256,1.114127,1.039934,-0.421911,0.302118,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094,WHITE,0
1,-1.019256,-0.897564,-0.961600,-0.570071,-1.053692,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,3.250718,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094,WHITE,0
2,-1.019256,-0.897564,-0.961600,-0.239893,0.648933,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,2.647779,-0.188538,-0.351233,-0.210781,0.479094,WHITE,0
3,-1.019256,-0.897564,1.039934,-0.257870,-0.071899,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,3.278612,-0.377675,-0.188538,-0.351233,-0.210781,0.479094,WHITE,0
4,-1.019256,-0.897564,1.039934,-0.484447,-0.171696,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094,WHITE,0


In [4]:
TestWhite = TestSet[TestSet['ethnicity'] == 'WHITE']
TestBlack = TestSet[TestSet['ethnicity'] == 'BLACK/AFRICAN AMERICAN']
TestHisp = TestSet[TestSet['ethnicity'] == 'HISPANIC/LATINO']
TestAsian = TestSet[TestSet['ethnicity'] == 'ASIAN']

In [5]:
xTestWhite = TestWhite.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestWhite = TestWhite['hospital_expire_flag'].reset_index()
yTestWhite = yTestWhite.drop(columns=['index'])
xTestBlack = TestBlack.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestBlack = TestBlack['hospital_expire_flag'].reset_index()
yTestBlack = yTestBlack.drop(columns=['index'])
xTestHisp = TestHisp.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestHisp = TestHisp['hospital_expire_flag'].reset_index()
yTestHisp = yTestHisp.drop(columns=['index'])
xTestAsian = TestAsian.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestAsian = TestAsian['hospital_expire_flag'].reset_index()
yTestAsian = yTestAsian.drop(columns=['index'])

Final edit to training and testing dataset

In [6]:
xTrain=xTrain.drop(columns=['ethnicity'])
xTest=xTest.drop(columns=['ethnicity'])
yTrain=yTrain['hospital_expire_flag']
yTest=yTest['hospital_expire_flag']

In [7]:
xTest.head()

,admission_location,insurance,marital_status,los,sofa_24hours,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT,ASIAN,BLACK/AFRICAN AMERICAN,HISPANIC/LATINO,WHITE
0,-1.019256,1.114127,1.039934,-0.421911,0.302118,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
1,-1.019256,-0.897564,-0.961600,-0.570071,-1.053692,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,3.250718,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
2,-1.019256,-0.897564,-0.961600,-0.239893,0.648933,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,2.647779,-0.188538,-0.351233,-0.210781,0.479094
3,-1.019256,-0.897564,1.039934,-0.257870,-0.071899,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,3.278612,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
4,-1.019256,-0.897564,1.039934,-0.484447,-0.171696,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094


In [8]:
xTestWhite.head()

,admission_location,insurance,marital_status,los,sofa_24hours,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT,ASIAN,BLACK/AFRICAN AMERICAN,HISPANIC/LATINO,WHITE
0,-1.019256,1.114127,1.039934,-0.421911,0.302118,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
1,-1.019256,-0.897564,-0.961600,-0.570071,-1.053692,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,3.250718,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
2,-1.019256,-0.897564,-0.961600,-0.239893,0.648933,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,2.647779,-0.188538,-0.351233,-0.210781,0.479094
3,-1.019256,-0.897564,1.039934,-0.257870,-0.071899,-0.176476,-0.18199,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,3.278612,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
4,-1.019256,-0.897564,1.039934,-0.484447,-0.171696,-0.176476,-0.18199,3.728497,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094


In [9]:
xTrain.head()

,admission_location,insurance,marital_status,los,sofa_24hours,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT,ASIAN,BLACK/AFRICAN AMERICAN,HISPANIC/LATINO,WHITE
0,0.981108,-0.897564,-0.961600,-0.551279,-0.669286,-0.176476,-0.181990,-0.268205,-0.173346,2.138243,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
1,-1.019256,1.114127,1.039934,-0.516693,0.215728,-0.176476,-0.181990,-0.268205,-0.173346,-0.467674,-0.759136,3.250718,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
2,-1.019256,1.114127,-0.961600,-0.300452,0.345313,-0.176476,-0.181990,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,2.647779,-0.188538,-0.351233,-0.210781,0.479094
3,0.981108,1.114127,1.039934,-0.502843,-0.890060,-0.176476,-0.181990,-0.268205,-0.173346,2.138243,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094
4,-1.019256,1.114127,-0.961600,-0.220871,0.002331,-0.176476,5.494809,-0.268205,-0.173346,-0.467674,-0.759136,-0.307624,-0.305007,-0.377675,-0.188538,-0.351233,-0.210781,0.479094


## Calculating overall AUROC w/ 95% Confidence Intervals for Chosen Model

In [10]:
model=RandomForestClassifier(n_estimators = 400, criterion='entropy', max_depth=20, min_samples_leaf=50, max_features=17, bootstrap=True, oob_score=True)
model.fit(xTrain, yTrain)
y_pred = model.predict_proba(xTest)

In [11]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTest, y_pred[:,1])))

Original ROC area: 0.853


In [12]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(yTest[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(yTest[indices], y_pred[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.848
Bootstrap #2 ROC area: 0.851
Bootstrap #3 ROC area: 0.858
Bootstrap #4 ROC area: 0.863
Bootstrap #5 ROC area: 0.857
Bootstrap #6 ROC area: 0.860
Bootstrap #7 ROC area: 0.856
Bootstrap #8 ROC area: 0.858
Bootstrap #9 ROC area: 0.853
Bootstrap #10 ROC area: 0.856
Bootstrap #11 ROC area: 0.845
Bootstrap #12 ROC area: 0.845
Bootstrap #13 ROC area: 0.860
Bootstrap #14 ROC area: 0.862
Bootstrap #15 ROC area: 0.853
Bootstrap #16 ROC area: 0.847
Bootstrap #17 ROC area: 0.854
Bootstrap #18 ROC area: 0.846
Bootstrap #19 ROC area: 0.835
Bootstrap #20 ROC area: 0.848
Bootstrap #21 ROC area: 0.841
Bootstrap #22 ROC area: 0.867
Bootstrap #23 ROC area: 0.855
Bootstrap #24 ROC area: 0.854
Bootstrap #25 ROC area: 0.850
Bootstrap #26 ROC area: 0.845
Bootstrap #27 ROC area: 0.848
Bootstrap #28 ROC area: 0.854
Bootstrap #29 ROC area: 0.858
Bootstrap #30 ROC area: 0.850
Bootstrap #31 ROC area: 0.857
Bootstrap #32 ROC area: 0.854
Bootstrap #33 ROC area: 0.848
Bootstrap #34 ROC a

Bootstrap #302 ROC area: 0.863
Bootstrap #303 ROC area: 0.840
Bootstrap #304 ROC area: 0.852
Bootstrap #305 ROC area: 0.841
Bootstrap #306 ROC area: 0.850
Bootstrap #307 ROC area: 0.853
Bootstrap #308 ROC area: 0.858
Bootstrap #309 ROC area: 0.842
Bootstrap #310 ROC area: 0.840
Bootstrap #311 ROC area: 0.842
Bootstrap #312 ROC area: 0.852
Bootstrap #313 ROC area: 0.847
Bootstrap #314 ROC area: 0.853
Bootstrap #315 ROC area: 0.842
Bootstrap #316 ROC area: 0.853
Bootstrap #317 ROC area: 0.859
Bootstrap #318 ROC area: 0.859
Bootstrap #319 ROC area: 0.854
Bootstrap #320 ROC area: 0.865
Bootstrap #321 ROC area: 0.861
Bootstrap #322 ROC area: 0.848
Bootstrap #323 ROC area: 0.856
Bootstrap #324 ROC area: 0.853
Bootstrap #325 ROC area: 0.852
Bootstrap #326 ROC area: 0.855
Bootstrap #327 ROC area: 0.853
Bootstrap #328 ROC area: 0.850
Bootstrap #329 ROC area: 0.857
Bootstrap #330 ROC area: 0.850
Bootstrap #331 ROC area: 0.851
Bootstrap #332 ROC area: 0.857
Bootstrap #333 ROC area: 0.853
Bootstra

Bootstrap #606 ROC area: 0.849
Bootstrap #607 ROC area: 0.853
Bootstrap #608 ROC area: 0.848
Bootstrap #609 ROC area: 0.856
Bootstrap #610 ROC area: 0.855
Bootstrap #611 ROC area: 0.861
Bootstrap #612 ROC area: 0.848
Bootstrap #613 ROC area: 0.862
Bootstrap #614 ROC area: 0.858
Bootstrap #615 ROC area: 0.863
Bootstrap #616 ROC area: 0.840
Bootstrap #617 ROC area: 0.862
Bootstrap #618 ROC area: 0.857
Bootstrap #619 ROC area: 0.855
Bootstrap #620 ROC area: 0.849
Bootstrap #621 ROC area: 0.863
Bootstrap #622 ROC area: 0.857
Bootstrap #623 ROC area: 0.847
Bootstrap #624 ROC area: 0.850
Bootstrap #625 ROC area: 0.854
Bootstrap #626 ROC area: 0.859
Bootstrap #627 ROC area: 0.838
Bootstrap #628 ROC area: 0.844
Bootstrap #629 ROC area: 0.853
Bootstrap #630 ROC area: 0.850
Bootstrap #631 ROC area: 0.862
Bootstrap #632 ROC area: 0.853
Bootstrap #633 ROC area: 0.859
Bootstrap #634 ROC area: 0.849
Bootstrap #635 ROC area: 0.858
Bootstrap #636 ROC area: 0.854
Bootstrap #637 ROC area: 0.861
Bootstra

Bootstrap #876 ROC area: 0.847
Bootstrap #877 ROC area: 0.853
Bootstrap #878 ROC area: 0.854
Bootstrap #879 ROC area: 0.857
Bootstrap #880 ROC area: 0.843
Bootstrap #881 ROC area: 0.845
Bootstrap #882 ROC area: 0.848
Bootstrap #883 ROC area: 0.853
Bootstrap #884 ROC area: 0.855
Bootstrap #885 ROC area: 0.839
Bootstrap #886 ROC area: 0.857
Bootstrap #887 ROC area: 0.846
Bootstrap #888 ROC area: 0.849
Bootstrap #889 ROC area: 0.860
Bootstrap #890 ROC area: 0.843
Bootstrap #891 ROC area: 0.851
Bootstrap #892 ROC area: 0.851
Bootstrap #893 ROC area: 0.852
Bootstrap #894 ROC area: 0.858
Bootstrap #895 ROC area: 0.851
Bootstrap #896 ROC area: 0.854
Bootstrap #897 ROC area: 0.844
Bootstrap #898 ROC area: 0.850
Bootstrap #899 ROC area: 0.834
Bootstrap #900 ROC area: 0.854
Bootstrap #901 ROC area: 0.841
Bootstrap #902 ROC area: 0.856
Bootstrap #903 ROC area: 0.848
Bootstrap #904 ROC area: 0.844
Bootstrap #905 ROC area: 0.857
Bootstrap #906 ROC area: 0.854
Bootstrap #907 ROC area: 0.857
Bootstra

In [13]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score: [0.842 - 0.863]


### Calculating 95% CI for individual races for Chosen Model

#### For White Patients Set

In [14]:
y_pred_white = model.predict_proba(xTestWhite)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestWhite, y_pred_white[:,1])))

Original ROC area: 0.854


In [15]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_white), len(y_pred_white))

    score = roc_auc_score(yTestWhite.to_numpy()[indices], y_pred_white[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.862
Bootstrap #2 ROC area: 0.851
Bootstrap #3 ROC area: 0.843
Bootstrap #4 ROC area: 0.844
Bootstrap #5 ROC area: 0.864
Bootstrap #6 ROC area: 0.845
Bootstrap #7 ROC area: 0.865
Bootstrap #8 ROC area: 0.844
Bootstrap #9 ROC area: 0.839
Bootstrap #10 ROC area: 0.856
Bootstrap #11 ROC area: 0.839
Bootstrap #12 ROC area: 0.853
Bootstrap #13 ROC area: 0.857
Bootstrap #14 ROC area: 0.851
Bootstrap #15 ROC area: 0.845
Bootstrap #16 ROC area: 0.864
Bootstrap #17 ROC area: 0.853
Bootstrap #18 ROC area: 0.862
Bootstrap #19 ROC area: 0.857
Bootstrap #20 ROC area: 0.853
Bootstrap #21 ROC area: 0.861
Bootstrap #22 ROC area: 0.845
Bootstrap #23 ROC area: 0.859
Bootstrap #24 ROC area: 0.852
Bootstrap #25 ROC area: 0.854
Bootstrap #26 ROC area: 0.853
Bootstrap #27 ROC area: 0.854
Bootstrap #28 ROC area: 0.861
Bootstrap #29 ROC area: 0.849
Bootstrap #30 ROC area: 0.865
Bootstrap #31 ROC area: 0.841
Bootstrap #32 ROC area: 0.859
Bootstrap #33 ROC area: 0.872
Bootstrap #34 ROC a

Bootstrap #308 ROC area: 0.841
Bootstrap #309 ROC area: 0.861
Bootstrap #310 ROC area: 0.867
Bootstrap #311 ROC area: 0.859
Bootstrap #312 ROC area: 0.855
Bootstrap #313 ROC area: 0.850
Bootstrap #314 ROC area: 0.861
Bootstrap #315 ROC area: 0.854
Bootstrap #316 ROC area: 0.853
Bootstrap #317 ROC area: 0.866
Bootstrap #318 ROC area: 0.859
Bootstrap #319 ROC area: 0.861
Bootstrap #320 ROC area: 0.847
Bootstrap #321 ROC area: 0.842
Bootstrap #322 ROC area: 0.855
Bootstrap #323 ROC area: 0.855
Bootstrap #324 ROC area: 0.857
Bootstrap #325 ROC area: 0.852
Bootstrap #326 ROC area: 0.844
Bootstrap #327 ROC area: 0.859
Bootstrap #328 ROC area: 0.862
Bootstrap #329 ROC area: 0.852
Bootstrap #330 ROC area: 0.863
Bootstrap #331 ROC area: 0.860
Bootstrap #332 ROC area: 0.866
Bootstrap #333 ROC area: 0.859
Bootstrap #334 ROC area: 0.848
Bootstrap #335 ROC area: 0.861
Bootstrap #336 ROC area: 0.840
Bootstrap #337 ROC area: 0.849
Bootstrap #338 ROC area: 0.862
Bootstrap #339 ROC area: 0.855
Bootstra

Bootstrap #621 ROC area: 0.860
Bootstrap #622 ROC area: 0.851
Bootstrap #623 ROC area: 0.854
Bootstrap #624 ROC area: 0.854
Bootstrap #625 ROC area: 0.855
Bootstrap #626 ROC area: 0.852
Bootstrap #627 ROC area: 0.845
Bootstrap #628 ROC area: 0.847
Bootstrap #629 ROC area: 0.858
Bootstrap #630 ROC area: 0.850
Bootstrap #631 ROC area: 0.860
Bootstrap #632 ROC area: 0.858
Bootstrap #633 ROC area: 0.848
Bootstrap #634 ROC area: 0.852
Bootstrap #635 ROC area: 0.856
Bootstrap #636 ROC area: 0.865
Bootstrap #637 ROC area: 0.863
Bootstrap #638 ROC area: 0.859
Bootstrap #639 ROC area: 0.862
Bootstrap #640 ROC area: 0.852
Bootstrap #641 ROC area: 0.845
Bootstrap #642 ROC area: 0.851
Bootstrap #643 ROC area: 0.846
Bootstrap #644 ROC area: 0.862
Bootstrap #645 ROC area: 0.851
Bootstrap #646 ROC area: 0.860
Bootstrap #647 ROC area: 0.856
Bootstrap #648 ROC area: 0.855
Bootstrap #649 ROC area: 0.857
Bootstrap #650 ROC area: 0.851
Bootstrap #651 ROC area: 0.858
Bootstrap #652 ROC area: 0.858
Bootstra

Bootstrap #936 ROC area: 0.851
Bootstrap #937 ROC area: 0.852
Bootstrap #938 ROC area: 0.857
Bootstrap #939 ROC area: 0.861
Bootstrap #940 ROC area: 0.846
Bootstrap #941 ROC area: 0.859
Bootstrap #942 ROC area: 0.854
Bootstrap #943 ROC area: 0.861
Bootstrap #944 ROC area: 0.862
Bootstrap #945 ROC area: 0.848
Bootstrap #946 ROC area: 0.846
Bootstrap #947 ROC area: 0.854
Bootstrap #948 ROC area: 0.858
Bootstrap #949 ROC area: 0.864
Bootstrap #950 ROC area: 0.845
Bootstrap #951 ROC area: 0.848
Bootstrap #952 ROC area: 0.841
Bootstrap #953 ROC area: 0.846
Bootstrap #954 ROC area: 0.856
Bootstrap #955 ROC area: 0.852
Bootstrap #956 ROC area: 0.844
Bootstrap #957 ROC area: 0.851
Bootstrap #958 ROC area: 0.851
Bootstrap #959 ROC area: 0.839
Bootstrap #960 ROC area: 0.853
Bootstrap #961 ROC area: 0.850
Bootstrap #962 ROC area: 0.847
Bootstrap #963 ROC area: 0.853
Bootstrap #964 ROC area: 0.849
Bootstrap #965 ROC area: 0.852
Bootstrap #966 ROC area: 0.852
Bootstrap #967 ROC area: 0.863
Bootstra

In [16]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score on White patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on White patients: [0.842 - 0.865]


#### For Black Patient Set

In [17]:
y_pred_black = model.predict_proba(xTestBlack)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestBlack, y_pred_black[:,1])))

Original ROC area: 0.886


In [18]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_black), len(y_pred_black))

    score = roc_auc_score(yTestBlack.to_numpy()[indices], y_pred_black[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.921
Bootstrap #2 ROC area: 0.881
Bootstrap #3 ROC area: 0.876
Bootstrap #4 ROC area: 0.847
Bootstrap #5 ROC area: 0.856
Bootstrap #6 ROC area: 0.893
Bootstrap #7 ROC area: 0.886
Bootstrap #8 ROC area: 0.858
Bootstrap #9 ROC area: 0.866
Bootstrap #10 ROC area: 0.882
Bootstrap #11 ROC area: 0.892
Bootstrap #12 ROC area: 0.908
Bootstrap #13 ROC area: 0.896
Bootstrap #14 ROC area: 0.906
Bootstrap #15 ROC area: 0.897
Bootstrap #16 ROC area: 0.924
Bootstrap #17 ROC area: 0.884
Bootstrap #18 ROC area: 0.892
Bootstrap #19 ROC area: 0.860
Bootstrap #20 ROC area: 0.850
Bootstrap #21 ROC area: 0.852
Bootstrap #22 ROC area: 0.884
Bootstrap #23 ROC area: 0.895
Bootstrap #24 ROC area: 0.881
Bootstrap #25 ROC area: 0.894
Bootstrap #26 ROC area: 0.906
Bootstrap #27 ROC area: 0.879
Bootstrap #28 ROC area: 0.888
Bootstrap #29 ROC area: 0.893
Bootstrap #30 ROC area: 0.873
Bootstrap #31 ROC area: 0.911
Bootstrap #32 ROC area: 0.881
Bootstrap #33 ROC area: 0.862
Bootstrap #34 ROC a

Bootstrap #279 ROC area: 0.861
Bootstrap #280 ROC area: 0.908
Bootstrap #281 ROC area: 0.889
Bootstrap #282 ROC area: 0.861
Bootstrap #283 ROC area: 0.916
Bootstrap #284 ROC area: 0.865
Bootstrap #285 ROC area: 0.884
Bootstrap #286 ROC area: 0.840
Bootstrap #287 ROC area: 0.892
Bootstrap #288 ROC area: 0.906
Bootstrap #289 ROC area: 0.867
Bootstrap #290 ROC area: 0.884
Bootstrap #291 ROC area: 0.885
Bootstrap #292 ROC area: 0.916
Bootstrap #293 ROC area: 0.897
Bootstrap #294 ROC area: 0.863
Bootstrap #295 ROC area: 0.910
Bootstrap #296 ROC area: 0.884
Bootstrap #297 ROC area: 0.878
Bootstrap #298 ROC area: 0.876
Bootstrap #299 ROC area: 0.899
Bootstrap #300 ROC area: 0.897
Bootstrap #301 ROC area: 0.884
Bootstrap #302 ROC area: 0.915
Bootstrap #303 ROC area: 0.906
Bootstrap #304 ROC area: 0.913
Bootstrap #305 ROC area: 0.893
Bootstrap #306 ROC area: 0.892
Bootstrap #307 ROC area: 0.912
Bootstrap #308 ROC area: 0.899
Bootstrap #309 ROC area: 0.880
Bootstrap #310 ROC area: 0.890
Bootstra

Bootstrap #578 ROC area: 0.904
Bootstrap #579 ROC area: 0.917
Bootstrap #580 ROC area: 0.864
Bootstrap #581 ROC area: 0.860
Bootstrap #582 ROC area: 0.871
Bootstrap #583 ROC area: 0.857
Bootstrap #584 ROC area: 0.887
Bootstrap #585 ROC area: 0.884
Bootstrap #586 ROC area: 0.845
Bootstrap #587 ROC area: 0.853
Bootstrap #588 ROC area: 0.888
Bootstrap #589 ROC area: 0.867
Bootstrap #590 ROC area: 0.880
Bootstrap #591 ROC area: 0.862
Bootstrap #592 ROC area: 0.904
Bootstrap #593 ROC area: 0.876
Bootstrap #594 ROC area: 0.878
Bootstrap #595 ROC area: 0.848
Bootstrap #596 ROC area: 0.878
Bootstrap #597 ROC area: 0.872
Bootstrap #598 ROC area: 0.873
Bootstrap #599 ROC area: 0.876
Bootstrap #600 ROC area: 0.882
Bootstrap #601 ROC area: 0.890
Bootstrap #602 ROC area: 0.887
Bootstrap #603 ROC area: 0.899
Bootstrap #604 ROC area: 0.892
Bootstrap #605 ROC area: 0.865
Bootstrap #606 ROC area: 0.894
Bootstrap #607 ROC area: 0.856
Bootstrap #608 ROC area: 0.875
Bootstrap #609 ROC area: 0.875
Bootstra

Bootstrap #911 ROC area: 0.853
Bootstrap #912 ROC area: 0.888
Bootstrap #913 ROC area: 0.908
Bootstrap #914 ROC area: 0.914
Bootstrap #915 ROC area: 0.904
Bootstrap #916 ROC area: 0.899
Bootstrap #917 ROC area: 0.909
Bootstrap #918 ROC area: 0.894
Bootstrap #919 ROC area: 0.885
Bootstrap #920 ROC area: 0.906
Bootstrap #921 ROC area: 0.889
Bootstrap #922 ROC area: 0.895
Bootstrap #923 ROC area: 0.850
Bootstrap #924 ROC area: 0.848
Bootstrap #925 ROC area: 0.876
Bootstrap #926 ROC area: 0.872
Bootstrap #927 ROC area: 0.921
Bootstrap #928 ROC area: 0.876
Bootstrap #929 ROC area: 0.870
Bootstrap #930 ROC area: 0.905
Bootstrap #931 ROC area: 0.875
Bootstrap #932 ROC area: 0.886
Bootstrap #933 ROC area: 0.869
Bootstrap #934 ROC area: 0.902
Bootstrap #935 ROC area: 0.857
Bootstrap #936 ROC area: 0.902
Bootstrap #937 ROC area: 0.833
Bootstrap #938 ROC area: 0.865
Bootstrap #939 ROC area: 0.894
Bootstrap #940 ROC area: 0.896
Bootstrap #941 ROC area: 0.892
Bootstrap #942 ROC area: 0.890
Bootstra

In [19]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Black patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Black patients: [0.847 - 0.92]


#### For Hispanic Patient Set

In [20]:
y_pred_hispanic = model.predict_proba(xTestHisp)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestHisp, y_pred_hispanic[:,1])))

Original ROC area: 0.803


In [21]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_hispanic), len(y_pred_hispanic))

    score = roc_auc_score(yTestHisp.to_numpy()[indices], y_pred_hispanic[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.808
Bootstrap #2 ROC area: 0.824
Bootstrap #3 ROC area: 0.844
Bootstrap #4 ROC area: 0.776
Bootstrap #5 ROC area: 0.854
Bootstrap #6 ROC area: 0.759
Bootstrap #7 ROC area: 0.798
Bootstrap #8 ROC area: 0.787
Bootstrap #9 ROC area: 0.730
Bootstrap #10 ROC area: 0.827
Bootstrap #11 ROC area: 0.822
Bootstrap #12 ROC area: 0.779
Bootstrap #13 ROC area: 0.838
Bootstrap #14 ROC area: 0.784
Bootstrap #15 ROC area: 0.791
Bootstrap #16 ROC area: 0.852
Bootstrap #17 ROC area: 0.826
Bootstrap #18 ROC area: 0.825
Bootstrap #19 ROC area: 0.855
Bootstrap #20 ROC area: 0.819
Bootstrap #21 ROC area: 0.735
Bootstrap #22 ROC area: 0.798
Bootstrap #23 ROC area: 0.800
Bootstrap #24 ROC area: 0.853
Bootstrap #25 ROC area: 0.795
Bootstrap #26 ROC area: 0.841
Bootstrap #27 ROC area: 0.851
Bootstrap #28 ROC area: 0.810
Bootstrap #29 ROC area: 0.758
Bootstrap #30 ROC area: 0.815
Bootstrap #31 ROC area: 0.770
Bootstrap #32 ROC area: 0.789
Bootstrap #33 ROC area: 0.841
Bootstrap #34 ROC a

Bootstrap #348 ROC area: 0.854
Bootstrap #349 ROC area: 0.824
Bootstrap #350 ROC area: 0.849
Bootstrap #351 ROC area: 0.816
Bootstrap #352 ROC area: 0.810
Bootstrap #353 ROC area: 0.839
Bootstrap #354 ROC area: 0.843
Bootstrap #355 ROC area: 0.785
Bootstrap #356 ROC area: 0.806
Bootstrap #357 ROC area: 0.799
Bootstrap #358 ROC area: 0.796
Bootstrap #359 ROC area: 0.773
Bootstrap #360 ROC area: 0.837
Bootstrap #361 ROC area: 0.799
Bootstrap #362 ROC area: 0.800
Bootstrap #363 ROC area: 0.796
Bootstrap #364 ROC area: 0.856
Bootstrap #365 ROC area: 0.793
Bootstrap #366 ROC area: 0.820
Bootstrap #367 ROC area: 0.815
Bootstrap #368 ROC area: 0.781
Bootstrap #369 ROC area: 0.864
Bootstrap #370 ROC area: 0.827
Bootstrap #371 ROC area: 0.832
Bootstrap #372 ROC area: 0.823
Bootstrap #373 ROC area: 0.834
Bootstrap #374 ROC area: 0.830
Bootstrap #375 ROC area: 0.830
Bootstrap #376 ROC area: 0.824
Bootstrap #377 ROC area: 0.802
Bootstrap #378 ROC area: 0.866
Bootstrap #379 ROC area: 0.819
Bootstra

Bootstrap #692 ROC area: 0.850
Bootstrap #693 ROC area: 0.812
Bootstrap #694 ROC area: 0.771
Bootstrap #695 ROC area: 0.865
Bootstrap #696 ROC area: 0.789
Bootstrap #697 ROC area: 0.816
Bootstrap #698 ROC area: 0.847
Bootstrap #699 ROC area: 0.835
Bootstrap #700 ROC area: 0.815
Bootstrap #701 ROC area: 0.768
Bootstrap #702 ROC area: 0.807
Bootstrap #703 ROC area: 0.841
Bootstrap #704 ROC area: 0.819
Bootstrap #705 ROC area: 0.841
Bootstrap #706 ROC area: 0.788
Bootstrap #707 ROC area: 0.819
Bootstrap #708 ROC area: 0.783
Bootstrap #709 ROC area: 0.807
Bootstrap #710 ROC area: 0.826
Bootstrap #711 ROC area: 0.862
Bootstrap #712 ROC area: 0.782
Bootstrap #713 ROC area: 0.810
Bootstrap #714 ROC area: 0.789
Bootstrap #715 ROC area: 0.851
Bootstrap #716 ROC area: 0.785
Bootstrap #717 ROC area: 0.808
Bootstrap #718 ROC area: 0.765
Bootstrap #719 ROC area: 0.843
Bootstrap #720 ROC area: 0.766
Bootstrap #721 ROC area: 0.830
Bootstrap #722 ROC area: 0.845
Bootstrap #723 ROC area: 0.841
Bootstra

Bootstrap #974 ROC area: 0.841
Bootstrap #975 ROC area: 0.779
Bootstrap #976 ROC area: 0.843
Bootstrap #977 ROC area: 0.827
Bootstrap #978 ROC area: 0.831
Bootstrap #979 ROC area: 0.790
Bootstrap #980 ROC area: 0.832
Bootstrap #981 ROC area: 0.799
Bootstrap #982 ROC area: 0.803
Bootstrap #983 ROC area: 0.844
Bootstrap #984 ROC area: 0.799
Bootstrap #985 ROC area: 0.803
Bootstrap #986 ROC area: 0.822
Bootstrap #987 ROC area: 0.840
Bootstrap #988 ROC area: 0.808
Bootstrap #989 ROC area: 0.785
Bootstrap #990 ROC area: 0.819
Bootstrap #991 ROC area: 0.790
Bootstrap #992 ROC area: 0.811
Bootstrap #993 ROC area: 0.850
Bootstrap #994 ROC area: 0.746
Bootstrap #995 ROC area: 0.805
Bootstrap #996 ROC area: 0.830
Bootstrap #997 ROC area: 0.882
Bootstrap #998 ROC area: 0.792
Bootstrap #999 ROC area: 0.825
Bootstrap #1000 ROC area: 0.757


In [22]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Hispanic patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Hispanic patients: [0.741 - 0.86]


#### For Asian Patient Set

In [23]:
y_pred_asian = model.predict_proba(xTestAsian)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestAsian, y_pred_asian[:,1])))

Original ROC area: 0.798


In [24]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_asian), len(y_pred_asian))

    score = roc_auc_score(yTestAsian.to_numpy()[indices], y_pred_asian[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.804
Bootstrap #2 ROC area: 0.864
Bootstrap #3 ROC area: 0.789
Bootstrap #4 ROC area: 0.779
Bootstrap #5 ROC area: 0.759
Bootstrap #6 ROC area: 0.802
Bootstrap #7 ROC area: 0.824
Bootstrap #8 ROC area: 0.815
Bootstrap #9 ROC area: 0.796
Bootstrap #10 ROC area: 0.805
Bootstrap #11 ROC area: 0.725
Bootstrap #12 ROC area: 0.768
Bootstrap #13 ROC area: 0.788
Bootstrap #14 ROC area: 0.725
Bootstrap #15 ROC area: 0.754
Bootstrap #16 ROC area: 0.731
Bootstrap #17 ROC area: 0.794
Bootstrap #18 ROC area: 0.722
Bootstrap #19 ROC area: 0.783
Bootstrap #20 ROC area: 0.822
Bootstrap #21 ROC area: 0.808
Bootstrap #22 ROC area: 0.856
Bootstrap #23 ROC area: 0.806
Bootstrap #24 ROC area: 0.810
Bootstrap #25 ROC area: 0.770
Bootstrap #26 ROC area: 0.835
Bootstrap #27 ROC area: 0.900
Bootstrap #28 ROC area: 0.856
Bootstrap #29 ROC area: 0.801
Bootstrap #30 ROC area: 0.710
Bootstrap #31 ROC area: 0.758
Bootstrap #32 ROC area: 0.816
Bootstrap #33 ROC area: 0.641
Bootstrap #34 ROC a

Bootstrap #311 ROC area: 0.811
Bootstrap #312 ROC area: 0.796
Bootstrap #313 ROC area: 0.843
Bootstrap #314 ROC area: 0.833
Bootstrap #315 ROC area: 0.799
Bootstrap #316 ROC area: 0.812
Bootstrap #317 ROC area: 0.842
Bootstrap #318 ROC area: 0.786
Bootstrap #319 ROC area: 0.776
Bootstrap #320 ROC area: 0.837
Bootstrap #321 ROC area: 0.797
Bootstrap #322 ROC area: 0.853
Bootstrap #323 ROC area: 0.839
Bootstrap #324 ROC area: 0.772
Bootstrap #325 ROC area: 0.829
Bootstrap #326 ROC area: 0.750
Bootstrap #327 ROC area: 0.831
Bootstrap #328 ROC area: 0.775
Bootstrap #329 ROC area: 0.790
Bootstrap #330 ROC area: 0.792
Bootstrap #331 ROC area: 0.827
Bootstrap #332 ROC area: 0.803
Bootstrap #333 ROC area: 0.810
Bootstrap #334 ROC area: 0.791
Bootstrap #335 ROC area: 0.782
Bootstrap #336 ROC area: 0.784
Bootstrap #337 ROC area: 0.772
Bootstrap #338 ROC area: 0.832
Bootstrap #339 ROC area: 0.847
Bootstrap #340 ROC area: 0.747
Bootstrap #341 ROC area: 0.827
Bootstrap #342 ROC area: 0.801
Bootstra

Bootstrap #576 ROC area: 0.837
Bootstrap #577 ROC area: 0.824
Bootstrap #578 ROC area: 0.833
Bootstrap #579 ROC area: 0.778
Bootstrap #580 ROC area: 0.843
Bootstrap #581 ROC area: 0.871
Bootstrap #582 ROC area: 0.763
Bootstrap #583 ROC area: 0.757
Bootstrap #584 ROC area: 0.773
Bootstrap #585 ROC area: 0.779
Bootstrap #586 ROC area: 0.781
Bootstrap #587 ROC area: 0.849
Bootstrap #588 ROC area: 0.777
Bootstrap #589 ROC area: 0.840
Bootstrap #590 ROC area: 0.801
Bootstrap #591 ROC area: 0.774
Bootstrap #592 ROC area: 0.865
Bootstrap #593 ROC area: 0.820
Bootstrap #594 ROC area: 0.789
Bootstrap #595 ROC area: 0.797
Bootstrap #596 ROC area: 0.848
Bootstrap #597 ROC area: 0.767
Bootstrap #598 ROC area: 0.782
Bootstrap #599 ROC area: 0.809
Bootstrap #600 ROC area: 0.839
Bootstrap #601 ROC area: 0.828
Bootstrap #602 ROC area: 0.789
Bootstrap #603 ROC area: 0.837
Bootstrap #604 ROC area: 0.773
Bootstrap #605 ROC area: 0.767
Bootstrap #606 ROC area: 0.807
Bootstrap #607 ROC area: 0.651
Bootstra

Bootstrap #878 ROC area: 0.812
Bootstrap #879 ROC area: 0.785
Bootstrap #880 ROC area: 0.768
Bootstrap #881 ROC area: 0.783
Bootstrap #882 ROC area: 0.816
Bootstrap #883 ROC area: 0.777
Bootstrap #884 ROC area: 0.857
Bootstrap #885 ROC area: 0.786
Bootstrap #886 ROC area: 0.755
Bootstrap #887 ROC area: 0.825
Bootstrap #888 ROC area: 0.787
Bootstrap #889 ROC area: 0.820
Bootstrap #890 ROC area: 0.774
Bootstrap #891 ROC area: 0.818
Bootstrap #892 ROC area: 0.780
Bootstrap #893 ROC area: 0.792
Bootstrap #894 ROC area: 0.808
Bootstrap #895 ROC area: 0.742
Bootstrap #896 ROC area: 0.803
Bootstrap #897 ROC area: 0.831
Bootstrap #898 ROC area: 0.811
Bootstrap #899 ROC area: 0.799
Bootstrap #900 ROC area: 0.812
Bootstrap #901 ROC area: 0.854
Bootstrap #902 ROC area: 0.820
Bootstrap #903 ROC area: 0.819
Bootstrap #904 ROC area: 0.823
Bootstrap #905 ROC area: 0.809
Bootstrap #906 ROC area: 0.786
Bootstrap #907 ROC area: 0.815
Bootstrap #908 ROC area: 0.779
Bootstrap #909 ROC area: 0.813
Bootstra

In [25]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Asian patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Asian patients: [0.722 - 0.864]


## Calculating overall AUROC w/ 95% Confidence Intervals for Logistic Regression Model

In [26]:
xTrainLog = xTrain['sofa_24hours'].to_numpy().reshape(-1,1)
xTestLog = xTest['sofa_24hours'].to_numpy().reshape(-1,1)

In [27]:
model=LogisticRegression()
model.fit(xTrainLog, yTrain)
y_pred = model.predict_proba(xTestLog)

In [28]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTest, y_pred[:,1])))

Original ROC area: 0.733


In [29]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(yTest[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(yTest[indices], y_pred[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.720
Bootstrap #2 ROC area: 0.726
Bootstrap #3 ROC area: 0.743
Bootstrap #4 ROC area: 0.751
Bootstrap #5 ROC area: 0.742
Bootstrap #6 ROC area: 0.742
Bootstrap #7 ROC area: 0.732
Bootstrap #8 ROC area: 0.743
Bootstrap #9 ROC area: 0.728
Bootstrap #10 ROC area: 0.740
Bootstrap #11 ROC area: 0.725
Bootstrap #12 ROC area: 0.730
Bootstrap #13 ROC area: 0.731
Bootstrap #14 ROC area: 0.731
Bootstrap #15 ROC area: 0.738
Bootstrap #16 ROC area: 0.740
Bootstrap #17 ROC area: 0.745
Bootstrap #18 ROC area: 0.727
Bootstrap #19 ROC area: 0.720
Bootstrap #20 ROC area: 0.714
Bootstrap #21 ROC area: 0.725
Bootstrap #22 ROC area: 0.755
Bootstrap #23 ROC area: 0.742
Bootstrap #24 ROC area: 0.731
Bootstrap #25 ROC area: 0.731
Bootstrap #26 ROC area: 0.722
Bootstrap #27 ROC area: 0.739
Bootstrap #28 ROC area: 0.732
Bootstrap #29 ROC area: 0.732
Bootstrap #30 ROC area: 0.736
Bootstrap #31 ROC area: 0.734
Bootstrap #32 ROC area: 0.736
Bootstrap #33 ROC area: 0.731
Bootstrap #34 ROC a

Bootstrap #270 ROC area: 0.720
Bootstrap #271 ROC area: 0.732
Bootstrap #272 ROC area: 0.730
Bootstrap #273 ROC area: 0.727
Bootstrap #274 ROC area: 0.734
Bootstrap #275 ROC area: 0.723
Bootstrap #276 ROC area: 0.729
Bootstrap #277 ROC area: 0.720
Bootstrap #278 ROC area: 0.737
Bootstrap #279 ROC area: 0.727
Bootstrap #280 ROC area: 0.733
Bootstrap #281 ROC area: 0.724
Bootstrap #282 ROC area: 0.732
Bootstrap #283 ROC area: 0.736
Bootstrap #284 ROC area: 0.732
Bootstrap #285 ROC area: 0.725
Bootstrap #286 ROC area: 0.733
Bootstrap #287 ROC area: 0.726
Bootstrap #288 ROC area: 0.748
Bootstrap #289 ROC area: 0.750
Bootstrap #290 ROC area: 0.718
Bootstrap #291 ROC area: 0.731
Bootstrap #292 ROC area: 0.726
Bootstrap #293 ROC area: 0.740
Bootstrap #294 ROC area: 0.728
Bootstrap #295 ROC area: 0.726
Bootstrap #296 ROC area: 0.716
Bootstrap #297 ROC area: 0.719
Bootstrap #298 ROC area: 0.716
Bootstrap #299 ROC area: 0.729
Bootstrap #300 ROC area: 0.730
Bootstrap #301 ROC area: 0.733
Bootstra

Bootstrap #549 ROC area: 0.728
Bootstrap #550 ROC area: 0.736
Bootstrap #551 ROC area: 0.726
Bootstrap #552 ROC area: 0.727
Bootstrap #553 ROC area: 0.732
Bootstrap #554 ROC area: 0.749
Bootstrap #555 ROC area: 0.720
Bootstrap #556 ROC area: 0.736
Bootstrap #557 ROC area: 0.724
Bootstrap #558 ROC area: 0.730
Bootstrap #559 ROC area: 0.718
Bootstrap #560 ROC area: 0.736
Bootstrap #561 ROC area: 0.740
Bootstrap #562 ROC area: 0.734
Bootstrap #563 ROC area: 0.746
Bootstrap #564 ROC area: 0.737
Bootstrap #565 ROC area: 0.737
Bootstrap #566 ROC area: 0.733
Bootstrap #567 ROC area: 0.721
Bootstrap #568 ROC area: 0.739
Bootstrap #569 ROC area: 0.739
Bootstrap #570 ROC area: 0.733
Bootstrap #571 ROC area: 0.733
Bootstrap #572 ROC area: 0.742
Bootstrap #573 ROC area: 0.722
Bootstrap #574 ROC area: 0.729
Bootstrap #575 ROC area: 0.726
Bootstrap #576 ROC area: 0.726
Bootstrap #577 ROC area: 0.731
Bootstrap #578 ROC area: 0.736
Bootstrap #579 ROC area: 0.722
Bootstrap #580 ROC area: 0.739
Bootstra

Bootstrap #816 ROC area: 0.738
Bootstrap #817 ROC area: 0.746
Bootstrap #818 ROC area: 0.741
Bootstrap #819 ROC area: 0.725
Bootstrap #820 ROC area: 0.729
Bootstrap #821 ROC area: 0.749
Bootstrap #822 ROC area: 0.730
Bootstrap #823 ROC area: 0.731
Bootstrap #824 ROC area: 0.741
Bootstrap #825 ROC area: 0.732
Bootstrap #826 ROC area: 0.750
Bootstrap #827 ROC area: 0.742
Bootstrap #828 ROC area: 0.737
Bootstrap #829 ROC area: 0.725
Bootstrap #830 ROC area: 0.746
Bootstrap #831 ROC area: 0.735
Bootstrap #832 ROC area: 0.735
Bootstrap #833 ROC area: 0.727
Bootstrap #834 ROC area: 0.740
Bootstrap #835 ROC area: 0.743
Bootstrap #836 ROC area: 0.735
Bootstrap #837 ROC area: 0.724
Bootstrap #838 ROC area: 0.730
Bootstrap #839 ROC area: 0.729
Bootstrap #840 ROC area: 0.729
Bootstrap #841 ROC area: 0.726
Bootstrap #842 ROC area: 0.750
Bootstrap #843 ROC area: 0.721
Bootstrap #844 ROC area: 0.734
Bootstrap #845 ROC area: 0.732
Bootstrap #846 ROC area: 0.724
Bootstrap #847 ROC area: 0.721
Bootstra

In [30]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score: [0.713 - 0.751]


### Selecting only SOFA score for individual race test dataset

In [31]:
xTestWhite = xTestWhite['sofa_24hours'].to_numpy().reshape(-1,1)
xTestBlack = xTestBlack['sofa_24hours'].to_numpy().reshape(-1,1)
xTestHisp = xTestHisp['sofa_24hours'].to_numpy().reshape(-1,1)
xTestAsian = xTestAsian['sofa_24hours'].to_numpy().reshape(-1,1)

#### For White Patient Set

In [32]:
y_pred_white = model.predict_proba(xTestWhite)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestWhite, y_pred_white[:,1])))

Original ROC area: 0.734


In [33]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_white), len(y_pred_white))

    score = roc_auc_score(yTestWhite.to_numpy()[indices], y_pred_white[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.720
Bootstrap #2 ROC area: 0.713
Bootstrap #3 ROC area: 0.731
Bootstrap #4 ROC area: 0.730
Bootstrap #5 ROC area: 0.749
Bootstrap #6 ROC area: 0.737
Bootstrap #7 ROC area: 0.750
Bootstrap #8 ROC area: 0.715
Bootstrap #9 ROC area: 0.708
Bootstrap #10 ROC area: 0.735
Bootstrap #11 ROC area: 0.732
Bootstrap #12 ROC area: 0.725
Bootstrap #13 ROC area: 0.744
Bootstrap #14 ROC area: 0.756
Bootstrap #15 ROC area: 0.723
Bootstrap #16 ROC area: 0.744
Bootstrap #17 ROC area: 0.734
Bootstrap #18 ROC area: 0.753
Bootstrap #19 ROC area: 0.742
Bootstrap #20 ROC area: 0.728
Bootstrap #21 ROC area: 0.732
Bootstrap #22 ROC area: 0.719
Bootstrap #23 ROC area: 0.736
Bootstrap #24 ROC area: 0.728
Bootstrap #25 ROC area: 0.738
Bootstrap #26 ROC area: 0.737
Bootstrap #27 ROC area: 0.736
Bootstrap #28 ROC area: 0.733
Bootstrap #29 ROC area: 0.737
Bootstrap #30 ROC area: 0.737
Bootstrap #31 ROC area: 0.722
Bootstrap #32 ROC area: 0.738
Bootstrap #33 ROC area: 0.754
Bootstrap #34 ROC a

Bootstrap #315 ROC area: 0.727
Bootstrap #316 ROC area: 0.718
Bootstrap #317 ROC area: 0.755
Bootstrap #318 ROC area: 0.739
Bootstrap #319 ROC area: 0.743
Bootstrap #320 ROC area: 0.745
Bootstrap #321 ROC area: 0.735
Bootstrap #322 ROC area: 0.724
Bootstrap #323 ROC area: 0.733
Bootstrap #324 ROC area: 0.730
Bootstrap #325 ROC area: 0.744
Bootstrap #326 ROC area: 0.723
Bootstrap #327 ROC area: 0.735
Bootstrap #328 ROC area: 0.735
Bootstrap #329 ROC area: 0.733
Bootstrap #330 ROC area: 0.741
Bootstrap #331 ROC area: 0.745
Bootstrap #332 ROC area: 0.746
Bootstrap #333 ROC area: 0.738
Bootstrap #334 ROC area: 0.739
Bootstrap #335 ROC area: 0.725
Bootstrap #336 ROC area: 0.730
Bootstrap #337 ROC area: 0.732
Bootstrap #338 ROC area: 0.748
Bootstrap #339 ROC area: 0.735
Bootstrap #340 ROC area: 0.735
Bootstrap #341 ROC area: 0.719
Bootstrap #342 ROC area: 0.747
Bootstrap #343 ROC area: 0.733
Bootstrap #344 ROC area: 0.728
Bootstrap #345 ROC area: 0.736
Bootstrap #346 ROC area: 0.740
Bootstra

Bootstrap #617 ROC area: 0.746
Bootstrap #618 ROC area: 0.735
Bootstrap #619 ROC area: 0.741
Bootstrap #620 ROC area: 0.745
Bootstrap #621 ROC area: 0.751
Bootstrap #622 ROC area: 0.736
Bootstrap #623 ROC area: 0.745
Bootstrap #624 ROC area: 0.735
Bootstrap #625 ROC area: 0.731
Bootstrap #626 ROC area: 0.735
Bootstrap #627 ROC area: 0.734
Bootstrap #628 ROC area: 0.723
Bootstrap #629 ROC area: 0.733
Bootstrap #630 ROC area: 0.734
Bootstrap #631 ROC area: 0.733
Bootstrap #632 ROC area: 0.737
Bootstrap #633 ROC area: 0.726
Bootstrap #634 ROC area: 0.737
Bootstrap #635 ROC area: 0.722
Bootstrap #636 ROC area: 0.741
Bootstrap #637 ROC area: 0.740
Bootstrap #638 ROC area: 0.737
Bootstrap #639 ROC area: 0.743
Bootstrap #640 ROC area: 0.721
Bootstrap #641 ROC area: 0.718
Bootstrap #642 ROC area: 0.729
Bootstrap #643 ROC area: 0.741
Bootstrap #644 ROC area: 0.746
Bootstrap #645 ROC area: 0.735
Bootstrap #646 ROC area: 0.742
Bootstrap #647 ROC area: 0.734
Bootstrap #648 ROC area: 0.739
Bootstra

Bootstrap #922 ROC area: 0.722
Bootstrap #923 ROC area: 0.738
Bootstrap #924 ROC area: 0.734
Bootstrap #925 ROC area: 0.732
Bootstrap #926 ROC area: 0.732
Bootstrap #927 ROC area: 0.740
Bootstrap #928 ROC area: 0.724
Bootstrap #929 ROC area: 0.749
Bootstrap #930 ROC area: 0.717
Bootstrap #931 ROC area: 0.734
Bootstrap #932 ROC area: 0.739
Bootstrap #933 ROC area: 0.740
Bootstrap #934 ROC area: 0.727
Bootstrap #935 ROC area: 0.732
Bootstrap #936 ROC area: 0.726
Bootstrap #937 ROC area: 0.740
Bootstrap #938 ROC area: 0.744
Bootstrap #939 ROC area: 0.748
Bootstrap #940 ROC area: 0.718
Bootstrap #941 ROC area: 0.737
Bootstrap #942 ROC area: 0.733
Bootstrap #943 ROC area: 0.744
Bootstrap #944 ROC area: 0.749
Bootstrap #945 ROC area: 0.715
Bootstrap #946 ROC area: 0.720
Bootstrap #947 ROC area: 0.731
Bootstrap #948 ROC area: 0.733
Bootstrap #949 ROC area: 0.742
Bootstrap #950 ROC area: 0.738
Bootstrap #951 ROC area: 0.731
Bootstrap #952 ROC area: 0.714
Bootstrap #953 ROC area: 0.750
Bootstra

In [34]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score on White patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on White patients: [0.717 - 0.75]


#### For Black Patient Set

In [35]:
y_pred_black = model.predict_proba(xTestBlack)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestBlack, y_pred_black[:,1])))

Original ROC area: 0.758


In [36]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_black), len(y_pred_black))

    score = roc_auc_score(yTestBlack.to_numpy()[indices], y_pred_black[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.876
Bootstrap #2 ROC area: 0.729
Bootstrap #3 ROC area: 0.732
Bootstrap #4 ROC area: 0.700
Bootstrap #5 ROC area: 0.721
Bootstrap #6 ROC area: 0.789
Bootstrap #7 ROC area: 0.761
Bootstrap #8 ROC area: 0.757
Bootstrap #9 ROC area: 0.736
Bootstrap #10 ROC area: 0.764
Bootstrap #11 ROC area: 0.756
Bootstrap #12 ROC area: 0.793
Bootstrap #13 ROC area: 0.750
Bootstrap #14 ROC area: 0.785
Bootstrap #15 ROC area: 0.745
Bootstrap #16 ROC area: 0.798
Bootstrap #17 ROC area: 0.803
Bootstrap #18 ROC area: 0.756
Bootstrap #19 ROC area: 0.744
Bootstrap #20 ROC area: 0.703
Bootstrap #21 ROC area: 0.721
Bootstrap #22 ROC area: 0.769
Bootstrap #23 ROC area: 0.744
Bootstrap #24 ROC area: 0.736
Bootstrap #25 ROC area: 0.785
Bootstrap #26 ROC area: 0.801
Bootstrap #27 ROC area: 0.732
Bootstrap #28 ROC area: 0.750
Bootstrap #29 ROC area: 0.766
Bootstrap #30 ROC area: 0.756
Bootstrap #31 ROC area: 0.792
Bootstrap #32 ROC area: 0.728
Bootstrap #33 ROC area: 0.700
Bootstrap #34 ROC a

Bootstrap #392 ROC area: 0.693
Bootstrap #393 ROC area: 0.747
Bootstrap #394 ROC area: 0.782
Bootstrap #395 ROC area: 0.716
Bootstrap #396 ROC area: 0.709
Bootstrap #397 ROC area: 0.788
Bootstrap #398 ROC area: 0.725
Bootstrap #399 ROC area: 0.816
Bootstrap #400 ROC area: 0.802
Bootstrap #401 ROC area: 0.806
Bootstrap #402 ROC area: 0.747
Bootstrap #403 ROC area: 0.800
Bootstrap #404 ROC area: 0.823
Bootstrap #405 ROC area: 0.717
Bootstrap #406 ROC area: 0.774
Bootstrap #407 ROC area: 0.795
Bootstrap #408 ROC area: 0.693
Bootstrap #409 ROC area: 0.808
Bootstrap #410 ROC area: 0.722
Bootstrap #411 ROC area: 0.715
Bootstrap #412 ROC area: 0.748
Bootstrap #413 ROC area: 0.760
Bootstrap #414 ROC area: 0.700
Bootstrap #415 ROC area: 0.767
Bootstrap #416 ROC area: 0.688
Bootstrap #417 ROC area: 0.810
Bootstrap #418 ROC area: 0.787
Bootstrap #419 ROC area: 0.754
Bootstrap #420 ROC area: 0.746
Bootstrap #421 ROC area: 0.766
Bootstrap #422 ROC area: 0.743
Bootstrap #423 ROC area: 0.736
Bootstra

Bootstrap #796 ROC area: 0.808
Bootstrap #797 ROC area: 0.754
Bootstrap #798 ROC area: 0.745
Bootstrap #799 ROC area: 0.767
Bootstrap #800 ROC area: 0.792
Bootstrap #801 ROC area: 0.786
Bootstrap #802 ROC area: 0.789
Bootstrap #803 ROC area: 0.769
Bootstrap #804 ROC area: 0.784
Bootstrap #805 ROC area: 0.723
Bootstrap #806 ROC area: 0.753
Bootstrap #807 ROC area: 0.869
Bootstrap #808 ROC area: 0.737
Bootstrap #809 ROC area: 0.755
Bootstrap #810 ROC area: 0.731
Bootstrap #811 ROC area: 0.803
Bootstrap #812 ROC area: 0.767
Bootstrap #813 ROC area: 0.708
Bootstrap #814 ROC area: 0.727
Bootstrap #815 ROC area: 0.774
Bootstrap #816 ROC area: 0.756
Bootstrap #817 ROC area: 0.737
Bootstrap #818 ROC area: 0.758
Bootstrap #819 ROC area: 0.736
Bootstrap #820 ROC area: 0.761
Bootstrap #821 ROC area: 0.741
Bootstrap #822 ROC area: 0.764
Bootstrap #823 ROC area: 0.758
Bootstrap #824 ROC area: 0.768
Bootstrap #825 ROC area: 0.743
Bootstrap #826 ROC area: 0.779
Bootstrap #827 ROC area: 0.769
Bootstra

In [37]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Black patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Black patients: [0.691 - 0.821]


#### For Hispanic Patient Set

In [38]:
y_pred_hispanic = model.predict_proba(xTestHisp)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestHisp, y_pred_hispanic[:,1])))

Original ROC area: 0.651


In [39]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_hispanic), len(y_pred_hispanic))

    score = roc_auc_score(yTestHisp.to_numpy()[indices], y_pred_hispanic[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.716
Bootstrap #2 ROC area: 0.634
Bootstrap #3 ROC area: 0.722
Bootstrap #4 ROC area: 0.605
Bootstrap #5 ROC area: 0.697
Bootstrap #6 ROC area: 0.660
Bootstrap #7 ROC area: 0.604
Bootstrap #8 ROC area: 0.619
Bootstrap #9 ROC area: 0.578
Bootstrap #10 ROC area: 0.555
Bootstrap #11 ROC area: 0.655
Bootstrap #12 ROC area: 0.630
Bootstrap #13 ROC area: 0.701
Bootstrap #14 ROC area: 0.641
Bootstrap #15 ROC area: 0.619
Bootstrap #16 ROC area: 0.722
Bootstrap #17 ROC area: 0.686
Bootstrap #18 ROC area: 0.613
Bootstrap #19 ROC area: 0.674
Bootstrap #20 ROC area: 0.669
Bootstrap #21 ROC area: 0.523
Bootstrap #22 ROC area: 0.565
Bootstrap #23 ROC area: 0.670
Bootstrap #24 ROC area: 0.699
Bootstrap #25 ROC area: 0.616
Bootstrap #26 ROC area: 0.697
Bootstrap #27 ROC area: 0.738
Bootstrap #28 ROC area: 0.653
Bootstrap #29 ROC area: 0.539
Bootstrap #30 ROC area: 0.669
Bootstrap #31 ROC area: 0.527
Bootstrap #32 ROC area: 0.576
Bootstrap #33 ROC area: 0.712
Bootstrap #34 ROC a

Bootstrap #479 ROC area: 0.638
Bootstrap #480 ROC area: 0.545
Bootstrap #481 ROC area: 0.601
Bootstrap #482 ROC area: 0.690
Bootstrap #483 ROC area: 0.708
Bootstrap #484 ROC area: 0.617
Bootstrap #485 ROC area: 0.718
Bootstrap #486 ROC area: 0.700
Bootstrap #487 ROC area: 0.711
Bootstrap #488 ROC area: 0.685
Bootstrap #489 ROC area: 0.687
Bootstrap #490 ROC area: 0.682
Bootstrap #491 ROC area: 0.679
Bootstrap #492 ROC area: 0.661
Bootstrap #493 ROC area: 0.636
Bootstrap #494 ROC area: 0.661
Bootstrap #495 ROC area: 0.645
Bootstrap #496 ROC area: 0.690
Bootstrap #497 ROC area: 0.687
Bootstrap #498 ROC area: 0.613
Bootstrap #499 ROC area: 0.637
Bootstrap #500 ROC area: 0.704
Bootstrap #501 ROC area: 0.637
Bootstrap #502 ROC area: 0.675
Bootstrap #503 ROC area: 0.760
Bootstrap #504 ROC area: 0.717
Bootstrap #505 ROC area: 0.550
Bootstrap #506 ROC area: 0.611
Bootstrap #507 ROC area: 0.662
Bootstrap #508 ROC area: 0.532
Bootstrap #509 ROC area: 0.668
Bootstrap #510 ROC area: 0.640
Bootstra

Bootstrap #958 ROC area: 0.646
Bootstrap #959 ROC area: 0.668
Bootstrap #960 ROC area: 0.656
Bootstrap #961 ROC area: 0.697
Bootstrap #962 ROC area: 0.687
Bootstrap #963 ROC area: 0.623
Bootstrap #964 ROC area: 0.534
Bootstrap #965 ROC area: 0.658
Bootstrap #966 ROC area: 0.585
Bootstrap #967 ROC area: 0.657
Bootstrap #968 ROC area: 0.653
Bootstrap #969 ROC area: 0.615
Bootstrap #970 ROC area: 0.651
Bootstrap #971 ROC area: 0.696
Bootstrap #972 ROC area: 0.473
Bootstrap #973 ROC area: 0.660
Bootstrap #974 ROC area: 0.709
Bootstrap #975 ROC area: 0.619
Bootstrap #976 ROC area: 0.655
Bootstrap #977 ROC area: 0.709
Bootstrap #978 ROC area: 0.687
Bootstrap #979 ROC area: 0.617
Bootstrap #980 ROC area: 0.717
Bootstrap #981 ROC area: 0.628
Bootstrap #982 ROC area: 0.602
Bootstrap #983 ROC area: 0.739
Bootstrap #984 ROC area: 0.628
Bootstrap #985 ROC area: 0.670
Bootstrap #986 ROC area: 0.695
Bootstrap #987 ROC area: 0.696
Bootstrap #988 ROC area: 0.577
Bootstrap #989 ROC area: 0.640
Bootstra

In [41]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Hispanic patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Hispanic patients: [0.545 - 0.75]


#### For Asian Patient Set

In [42]:
y_pred_asian = model.predict_proba(xTestAsian)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestAsian, y_pred_asian[:,1])))

Original ROC area: 0.729


In [43]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_asian), len(y_pred_asian))

    score = roc_auc_score(yTestAsian.to_numpy()[indices], y_pred_asian[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

Bootstrap #1 ROC area: 0.773
Bootstrap #2 ROC area: 0.775
Bootstrap #3 ROC area: 0.613
Bootstrap #4 ROC area: 0.741
Bootstrap #5 ROC area: 0.703
Bootstrap #6 ROC area: 0.762
Bootstrap #7 ROC area: 0.774
Bootstrap #8 ROC area: 0.716
Bootstrap #9 ROC area: 0.739
Bootstrap #10 ROC area: 0.706
Bootstrap #11 ROC area: 0.748
Bootstrap #12 ROC area: 0.755
Bootstrap #13 ROC area: 0.732
Bootstrap #14 ROC area: 0.678
Bootstrap #15 ROC area: 0.712
Bootstrap #16 ROC area: 0.588
Bootstrap #17 ROC area: 0.667
Bootstrap #18 ROC area: 0.631
Bootstrap #19 ROC area: 0.704
Bootstrap #20 ROC area: 0.750
Bootstrap #21 ROC area: 0.761
Bootstrap #22 ROC area: 0.801
Bootstrap #23 ROC area: 0.752
Bootstrap #24 ROC area: 0.697
Bootstrap #25 ROC area: 0.706
Bootstrap #26 ROC area: 0.765
Bootstrap #27 ROC area: 0.817
Bootstrap #28 ROC area: 0.858
Bootstrap #29 ROC area: 0.705
Bootstrap #30 ROC area: 0.730
Bootstrap #31 ROC area: 0.750
Bootstrap #32 ROC area: 0.771
Bootstrap #33 ROC area: 0.640
Bootstrap #34 ROC a

Bootstrap #519 ROC area: 0.791
Bootstrap #520 ROC area: 0.685
Bootstrap #521 ROC area: 0.746
Bootstrap #522 ROC area: 0.755
Bootstrap #523 ROC area: 0.822
Bootstrap #524 ROC area: 0.697
Bootstrap #525 ROC area: 0.706
Bootstrap #526 ROC area: 0.796
Bootstrap #527 ROC area: 0.783
Bootstrap #528 ROC area: 0.729
Bootstrap #529 ROC area: 0.745
Bootstrap #530 ROC area: 0.732
Bootstrap #531 ROC area: 0.785
Bootstrap #532 ROC area: 0.727
Bootstrap #533 ROC area: 0.706
Bootstrap #534 ROC area: 0.729
Bootstrap #535 ROC area: 0.806
Bootstrap #536 ROC area: 0.784
Bootstrap #537 ROC area: 0.753
Bootstrap #538 ROC area: 0.724
Bootstrap #539 ROC area: 0.742
Bootstrap #540 ROC area: 0.665
Bootstrap #541 ROC area: 0.772
Bootstrap #542 ROC area: 0.729
Bootstrap #543 ROC area: 0.709
Bootstrap #544 ROC area: 0.739
Bootstrap #545 ROC area: 0.717
Bootstrap #546 ROC area: 0.754
Bootstrap #547 ROC area: 0.706
Bootstrap #548 ROC area: 0.738
Bootstrap #549 ROC area: 0.782
Bootstrap #550 ROC area: 0.642
Bootstra

Bootstrap #786 ROC area: 0.730
Bootstrap #787 ROC area: 0.732
Bootstrap #788 ROC area: 0.708
Bootstrap #789 ROC area: 0.724
Bootstrap #790 ROC area: 0.732
Bootstrap #791 ROC area: 0.756
Bootstrap #792 ROC area: 0.766
Bootstrap #793 ROC area: 0.771
Bootstrap #794 ROC area: 0.705
Bootstrap #795 ROC area: 0.743
Bootstrap #796 ROC area: 0.781
Bootstrap #797 ROC area: 0.767
Bootstrap #798 ROC area: 0.641
Bootstrap #799 ROC area: 0.682
Bootstrap #800 ROC area: 0.731
Bootstrap #801 ROC area: 0.722
Bootstrap #802 ROC area: 0.692
Bootstrap #803 ROC area: 0.790
Bootstrap #804 ROC area: 0.712
Bootstrap #805 ROC area: 0.695
Bootstrap #806 ROC area: 0.724
Bootstrap #807 ROC area: 0.762
Bootstrap #808 ROC area: 0.759
Bootstrap #809 ROC area: 0.654
Bootstrap #810 ROC area: 0.765
Bootstrap #811 ROC area: 0.670
Bootstrap #812 ROC area: 0.727
Bootstrap #813 ROC area: 0.795
Bootstrap #814 ROC area: 0.669
Bootstrap #815 ROC area: 0.699
Bootstrap #816 ROC area: 0.814
Bootstrap #817 ROC area: 0.656
Bootstra

In [44]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Asian patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

Confidence interval for the score on Asian patients: [0.633 - 0.813]
